# MNIST & Fashion CNN

Before working on the MIL, we'd like to produce a CNN which can properly classify MNIST and Fashion MNIST. Because the fashion MNIST is more difficult, we'll start with it.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(42)

N_CLASSES = 10

print(tf.__version__)

2.4.1


Helper functions to load the datasets.

In [2]:
def preprocess(dataset):
  (x_train, y_train), (x_test, y_test) = dataset

  # Standardize
  x_train = x_train[..., np.newaxis] / 255 #tf.image.per_image_standardization(x_train[..., np.newaxis]).numpy()
  x_test = x_test[..., np.newaxis] / 255 #tf.image.per_image_standardization(x_test[..., np.newaxis]).numpy()

  # One hot
  y_train = tf.one_hot(y_train, N_CLASSES).numpy()
  y_test = tf.one_hot(y_test, N_CLASSES).numpy()

  return (x_train, y_train), (x_test, y_test)

def load_fashion():
  return preprocess(tf.keras.datasets.fashion_mnist.load_data())

def load_mnist():
  return preprocess(tf.keras.datasets.mnist.load_data())

Helper function to train the model.

In [3]:
def evaluate(create_model, load_dataset, n_folds=5):
  (x_train, y_train), (x_test, y_test) = load_dataset()
  print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

  model = create_model()

  return model.fit(
      x_train, y_train,
      callbacks=[ tf.keras.callbacks.EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      ) ],
      validation_data=(x_test, y_test),
      epochs=10,
      batch_size=64
  )

In [4]:
def define_model():
  model = tf.keras.Sequential([
      layers.Input((28, 28, 1)),
      layers.Conv2D(64, 2, activation='relu', padding='same'),
      layers.MaxPool2D(2),
      layers.Conv2D(32, 3, activation='relu'),
      layers.MaxPool2D(2),
      layers.Dropout(0.3),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(N_CLASSES, activation='softmax'),
  ])

  model.compile(
      optimizer=tf.optimizers.Adam(),
      loss=tf.losses.CategoricalCrossentropy(),
      metrics=[
          'accuracy',
          tfa.metrics.F1Score(N_CLASSES, name='f1'),
      ],
  )
  return model

print(define_model().summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2

In [5]:
evaluate(define_model, load_fashion)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)
Epoch 1/10
938/938 [==============================] - 34s 35ms/step - loss: 0.7630 - accuracy: 0.7228 - f1: 0.7170 - val_loss: 0.3564 - val_accuracy: 0.8717 - val_f1: 0.8695
Epoch 2/10
938/938 [==============================] - 31s 33ms/step - loss: 0.3764 - accuracy: 0.8620 - f1: 0.8600 - val_loss: 0.3148 - val_accuracy: 0.8840 - val_f1: 0.8817
Epoch 3/10
938/938 [==============================] - 32s 34ms/step - loss: 0.3307 - accuracy: 0.8784 - f1: 0.8780 - val_loss: 0.2749 - val_accuracy: 0.8982 - val_f1: 0.8976
Epoch 4/10
938/938 [==============================] - 31s 34ms/step - loss: 0.2976 - accuracy: 0.8923 - f1: 0.8914 - val_loss: 0.2638 - val_accuracy: 0.9007 - val_f1: 0.9022
Epoch 5/10
938/938 [==============================] - 32s 34ms/step - loss: 0.2782 - accuracy: 0.8976 - f1: 0.8973 - val_loss: 0.2575 - val_accuracy: 0.9047 - val_f1: 0.9031
Epoch 6/10
938/938 [==============================] - 32s 34ms/step 

In [6]:
evaluate(define_model, load_mnist)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)
Epoch 1/10
938/938 [==============================] - 35s 37ms/step - loss: 0.5053 - accuracy: 0.8346 - f1: 0.8320 - val_loss: 0.0541 - val_accuracy: 0.9834 - val_f1: 0.9833
Epoch 2/10
938/938 [==============================] - 33s 35ms/step - loss: 0.0924 - accuracy: 0.9713 - f1: 0.9712 - val_loss: 0.0375 - val_accuracy: 0.9888 - val_f1: 0.9887
Epoch 3/10
938/938 [==============================] - 33s 35ms/step - loss: 0.0700 - accuracy: 0.9776 - f1: 0.9774 - val_loss: 0.0315 - val_accuracy: 0.9889 - val_f1: 0.9889
Epoch 4/10
938/938 [==============================] - 33s 36ms/step - loss: 0.0575 - accuracy: 0.9818 - f1: 0.9816 - val_loss: 0.0316 - val_accuracy: 0.9905 - val_f1: 0.9905
Epoch 5/10
938/938 [==============================] - 35s 38ms/step - loss: 0.0495 - accuracy: 0.9842 - f1: 0.9841 - val_loss: 0.0324 - val_accuracy: 0.9888 - val_f1: 0.9887
Epoch 6/10
938/938 [==============================] - 34s 36ms/step 